## Allele filp QC YML generator
This module takes in a table of sumstat, with the columns: #chr, theme1, theme2, theme3 and each rows as 1 chr and the sumstat of corresponding chr and generate a list of yml to be used

In [ ]:
[global]
# a table of sumstat, with the columns: #chr, theme1, theme2, theme3 and each rows as 1 chr and the sumstat of corresponding chr
parameter: sumstat_list_path = path
parameter: wd = path
import pandas as pd
sumstat_list = pd.read_csv(sumstat_list_path, sep = "\t")
sumstat_inv = sumstat_list.values.tolist()
name = "_".join(sumstat_list.columns.values[1:len(sumstat_list.columns.values)].tolist())

## Assuming all the input sumstat are using the same header
parameter: CHR = "chr"
parameter: POS = "pos"
parameter: A0 = "ref" 
parameter: A1 = "alt"
parameter: SNP = "variant_id"
parameter: STAT = "beta"
parameter: SE: = "se"
parameter: P = "pval"
parameter: KEEP_AMBIGUOUS = "True"


In [ ]:
[yml_generator]
input: for_each  = "sumstat_inv"
output: f'{wd:a}/{name}.{_sumstat_inv[0]}/{name}.{_sumstat_inv[0]}.yml'
bash: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
        echo '
        INPUT:' > $[_output]

        for i in $[" ".join(_sumstat_inv[1:len(_sumstat_inv)])] ; do 
        echo "  
            - $i:
                CHR: $[CHR]
                POS: $[POS]
                A0: $[REF]
                A1: $[ALT]
                SNP: $[SNP]
                STAT: $[BETA]
                SE: $[SE]
                P: $[P]" >> $[_output];
        done 
        
        echo "
        Target:
            - $[_sumstat_inv[1]]:
                CHR: $[CHR]
                POS: $[POS]
                A0: $[REF]
                A1: $[ALT]
                SNP: $[SNP]
                STAT: $[BETA]
                SE: $[SE]
                P: $[P]
        OUTPUT: $[_output:d]
        KEEP_AMBIGUOUS: $[KEEP_AMBIGUOUS]" >> $[_output]

In [ ]:
[yml_list]
input: output_from("generator"),group_by = "all"
output: f'{wd:a}/yml_list.txt'
python: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
        import pandas as pd
        yml_df = pd.DataFrame({"#chr" : pd.read_csv($[sumstat_list_path:r],sep = "\t")["#chr"].values.tolist()  ,  "dir"  : [$[_input:r,]] })
        yml_df.to_csv("$[_output]",sep = "\t",index = 0)